# Potencial Ornitológico Fueguino
### **Autor:** Pablo Jusim

# Script de preprocesamiento de observaciones de eBird

![title](../data/external/logo_ebird.png)

Se realizan verificaciones antes de obtener el dataframe final.

La salida de este script será un dataframe de pandas donde cada fila representará una observación de eBird.

El dataframe final tendrá las siguientes **columnas**: id_obs, id_celda, especie, familia, orden, fecha

## Importaciones

In [ ]:
import pandas as pd
from IPython.display import display

## Carga de datos

**Nota para el usuario del notebook:** EL archivo "obs_ebird.txt" pesa 166 mb, por lo que es muy pesado para subirlo a GitHub. El mismo puede ser descargado de [google drive](https://drive.google.com/file/d/1Dlc4CDrUaHSdlqO_qWHAMROhHPoJIOhG/view?usp=sharing).

In [ ]:
archivo = '../data/external/obs_ebird.txt'
try:
    df_base = pd.read_csv(archivo, sep='\t')
    display(df_base.head())
except FileNotFoundError:
    print('No se encuentra el archivo en la ruta ' + archivo)

## Limpieza de datos

### Revisión y eliminación de columnas no útiles

In [ ]:
# Conservo únicamente las columnas necesarias y renombro según el formato de iNaturalist
df_filtrado = df_base[['COMMON NAME', 'SCIENTIFIC NAME', 'LATITUDE', 'LONGITUDE', 'OBSERVATION DATE']].rename(columns={
    'COMMON NAME': 'common_name',
    'SCIENTIFIC NAME': 'scientific_name',
    'LATITUDE': 'latitude',
    'LONGITUDE': 'longitude',
    'OBSERVATION DATE': 'observed_on'
})


##### Ver resumen de valores

In [ ]:
# Ver la cantidad de datos faltantes en cada fila
print(df_filtrado.isnull().sum())

In [ ]:
# Reviso puntos extremos de las coordenadas
df_filtrado.describe()

## Exportar el data frame resultante

In [ ]:
df_filtrado.to_csv('../data/raw/data_ebird.csv', index=False)